<div style="height:200px;">
    <img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3004/logos/header.png">
</div>

<h1 id="data" style="color:white; background:black; border:1px dashed;"> 
    <center>Data
        <a class="anchor-link" href="#data" target="_self">¶</a>
    </center>
</h1>

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pandas as pd
import numpy as np
import math

In [ ]:
path = "/kaggle/input/digit-recognizer/"
df = pd.read_csv(path + "train.csv")
df = shuffle(df)
df.head()

In [ ]:
features = df.iloc[:,1:].values.reshape(len(df), 28, 28)
labels = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(
                                    features, labels, test_size=0.33, random_state=42)

<h1 id="implementation" style="color:white; background:black; border:1px dashed;"> 
    <center>Implementation
        <a class="anchor-link" href="#implementation" target="_self">¶</a>
    </center>
</h1>

### Convolution 3x3

In [ ]:
class Conv3x3:
    '''
        A Convolution layer using 3x3 filters.
    '''

    def __init__(self, num_filters):
        self.num_filters = num_filters

        # filters is a 3d array with dimensions (num_filters, 3, 3)
        # We divide by 9 to reduce the variance of our initial values
        self.filters = np.random.randn(num_filters, 3, 3) / 9

    def iterate_regions(self, image):
        '''
        Generates all possible 3x3 image regions using valid padding.
        - image is a 2d numpy array.
        '''
        h, w = image.shape

        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the conv layer using the given input.
        Returns a 3d numpy array with dimensions (h, w, num_filters).
        - input is a 2d numpy array
        '''
        self.last_input = input

        h, w = input.shape
        output = np.zeros((h - 2, w - 2, self.num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

        return output

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the conv layer.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        d_L_d_filters = np.zeros(self.filters.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

        # Update filters
        self.filters -= learn_rate * d_L_d_filters

        # We aren't returning anything here since we use Conv3x3 as the first layer in our CNN.
        # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
        # other layer in our CNN.
        return None


### MaxPool 2x2

In [ ]:
class MaxPool2:
    '''
      A Max Pooling layer using a pool size of 2
    '''

    def iterate_regions(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the maxpool layer using the given input.
        Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
        - input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        self.last_input = input

        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
        '''
        Performs a backward pass of the maxpool layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        '''
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                        if im_region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]
                            
        return d_L_d_input

### Write the Softmax class

In [ ]:
class Softmax:
    '''
        A standard fully-connected layer with softmax activation.
    '''

    def __init__(self, input_len, nodes):
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        '''
        Performs a forward pass of the softmax layer using the given input.
        Returns a 1d numpy array containing the respective probability values.
        - input can be any array with any dimensions.
        '''
        self.last_input_shape = input.shape

        input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the softmax layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        # We know only 1 element of d_L_d_out will be nonzero
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            # e^totals
            t_exp = np.exp(self.last_totals)

            # Sum of all e^totals
            S = np.sum(t_exp)

            # Gradients of out[i] against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            # Gradients of totals against weights/biases/input
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t

            # Gradients of loss against weights/biases/input
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            # Update weights / biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

        return d_L_d_inputs.reshape(self.last_input_shape)

<h1 id="training" style="color:white; background:black; border:1px dashed;"> 
    <center>Training
        <a class="anchor-link" href="#training" target="_self">¶</a>
    </center>
</h1>

In [ ]:
def forward(image, label):
    '''
    Completes a forward pass of the CNN and calculates the accuracy and
    cross-entropy loss.
    - image is a 2d numpy array
    - label is a digit
    '''
    # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
    # to work with. This is standard practice.
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)

    # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0

    return out, loss, acc

def train(im, label, lr=.005):
    '''
    Completes a full training step on the given image and label.
    Returns the cross-entropy loss and accuracy.
    - image is a 2d numpy array
    - label is a digit
    - lr is the learning rate
    '''
    # Forward
    out, loss, acc = forward(im, label)

    # Calculate initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]

    # Backprop
    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)

    return loss, acc

In [ ]:
# 28x28x1 -> 26x26x8
conv = Conv3x3(8)

# 26x26x8 -> 13x13x8
pool = MaxPool2()

# 13x13x8 -> 10
softmax = Softmax(13 * 13 * 8, 10)

In [ ]:
for epoch in range(1):
    print('--- Epoch %d ---' % (epoch + 1))

    # shuffle
    X_train, y_train = shuffle(X_train, y_train)

    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(X_train, y_train)):
        if i > 0 and i % 2000 == 1999:
            print('[Step {:5d}] Past 2000 steps: Average Loss {:.3f} | Accuracy: {:2.2f}%'
                   .format(i + 1, loss / 2000, num_correct / 20))
            
            loss = 0
            num_correct = 0

        l, acc = train(im, label)
        loss += l
        num_correct += acc

In [ ]:
print('--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(X_test, y_test):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

num_tests = len(X_test)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

<h1 id="prediction" style="color:white; background:black; border:1px dashed;"> 
    <center>Prediction
        <a class="anchor-link" href="#prediction" target="_self">¶</a>
    </center>
</h1>

In [ ]:
def predict(x):
    '''
        Function to predict label
        return the softmax for each
    '''
    out, _, _ = forward(im, label)
    return out

<h1 id="reference" style="color:white; background:black; border:1px dashed;"> 
    <center>Reference
        <a class="anchor-link" href="#reference" target="_self">¶</a>
    </center>
</h1>

I have removed my old code and added [Victor Zhou code](https://victorzhou.com/blog/intro-to-cnns-part-1/) implementation just because it's a beauty.

<h1 id="submission" style="color:white; background:black; border:1px dashed;"> 
    <center>Submission
        <a class="anchor-link" href="#submission" target="_self">¶</a>
    </center>
</h1>

In [ ]:
# load submission data
df_test = pd.read_csv(path + "test.csv")
subm_arr = df_test.values.reshape(len(df_test), 28, 28)

In [ ]:
def subm_predict(subm_arr):
    '''
        Function to predict submission prediction
        return the max for each prediction
    '''
    result = []

    for i in tqdm(range(len(subm_arr))):
        result.append([i + 1, np.argmax(predict(subm_arr[i]))])
        
    return result

# predict submission data
result = subm_predict(subm_arr)

In [ ]:
df_final = pd.DataFrame(data=result, columns=["ImageId", "Label"])
df_final.to_csv('submission.csv', index=False)